In [ ]:
r = requests.get('https://forum.awd.ru/viewtopic.php?f=521&t=397637&start=120', headers = headers)

sp = bs(r.text, 'html.parser')

sp.find_all('div', 'content')



In [2]:
import pandas as pd
import seaborn as sns
import requests
from bs4 import BeautifulSoup as bs
import dfply
import re
import numpy as np


Функция для получения нужных страниц форума

In [3]:
def parse_forum(url, starts, headers):
    result = []
    for start in starts:
        request = requests.get(url = url + str(start), headers = headers)
        result.append(bs(request.text, 'html.parser'))
    return result

Функция для спарсивания нужных кусков страницы (del нужно, потому что на каждой странице вверху есть опрос, который тоже спарсивается, но не нужен)

In [4]:
def parse_soups(soups, tag, class_, delete):
    result = []
    for el in soups:
        content = el.find_all(tag, class_ = class_)
        del content[delete]
        for elem in content:
            result.append(elem.text.strip())
    return result

In [5]:
starts = [0,20, 40, 60, 80, 100, 120, 140]
headers = {
    'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/15.5 Safari/605.1.15',
    'cookie': '_ga_W0JDW2V29Y=GS1.1.1661181837.2.1.1661184512.60.0.0; _ga=GA1.2.1025230771.1661155454; _gid=GA1.2.721157209.1661155454; style_cookie=null; phpbb3_alft2_k=; phpbb3_alft2_sid=fa5f9f17fa55d5a1a82d62c62159899a; phpbb3_alft2_u=1'
}

soups = parse_forum('https://forum.awd.ru/viewtopic.php?f=521&t=397637&start=', starts, headers)

In [6]:
contents = parse_soups(soups, 'div', 'content', 0)
authors = parse_soups(soups, 'p', 'author', 0)

In [7]:
print(len(contents))
print(len(authors))

156
156


In [ ]:
contents[120:131]

Функция для парсинга уже конкретных данных по регулярному выражению
Айди нужен, чтобы потом вручную прописать данные, которые не спарсятся, и не потерять порядок людей

In [94]:
def get_data(regex, list, group_num = 2, hard = False, hard_regex = '(5\.)(.*)'):
    result = []
    for id, value in enumerate(list):
        try:
            reg = re.search(regex, value).group(group_num).strip()
            
            if hard == True:
                for i in range(3):
                    try:
                        reg = re.search(hard_regex, reg).group(2).strip()
                    except:
                        break
                        
                        
            result.append(reg)
        except:
            print("id = " + str(id) + "__Не пон " + str(value) + "\n___________")
            result.append(str(id) + " NaN")
    return result

In [ ]:
city = get_data('(1\.)(.*?)(2\.)', contents)
city

In [ ]:
visa_center = get_data('(2\.)(.*?)(3\.)', contents) 
visa_center

In [ ]:
date = get_data('\d\d\s\w{2,4}\s\d\d\d\d', authors, 0)
date

In [ ]:
visa_history = get_data('(4\.)(.*?)(5\.)', contents)
visa_history

In [ ]:
visa_limit = get_data('(5\.)(.+?)(6\.)', contents, hard = True)
visa_limit

In [140]:
date = get_data('\d\d\s\w{2,4}\s\d\d\d\d', authors, 0)
city = get_data('(1\.)(.*?)(2\.)', contents)
visa_center = get_data('(2\.)(.*?)(3\.)', contents) 
raw_history = get_data('(4\.)(.*?)(5\.)', contents)
raw_limit = get_data('(5\.)(.+?)(6\.)', contents, hard = True)

del date[0]
del city[0]
del visa_center[0]
del raw_history[0]
del raw_limit[0]

id = 150__Не пон эМСи » Вчера, 09:13
___________
id = 151__Не пон elia_bau » Вчера, 10:58
___________
id = 152__Не пон Marta77 » Вчера, 13:29
___________
id = 153__Не пон mr.drug » Вчера, 14:55
___________
id = 154__Не пон lenukass » Вчера, 18:11
___________
id = 155__Не пон IllustriousKane » Сегодня, 11:25
___________
id = 45__Не пон Калининград Вц 7 дней 3 дня Германия, 3 шенгена пол пол года, последний Греция под даты 1 годаТуризм Прикладывала справку на 160 тыс, бронь отеля, пфайзер 3 шота Рада пипец, после Греции думала будет тоже самое
___________
id = 93__Не пон 1 Москва 2 Визаметрик Киевская 3 21.07-03.08 (на визе 01.08) 4 греческие 3 года у мужа, у меня и детей 1,5 греческие. Вьетнам. Паспорт сильно в печатях за последние 3 года. 5 4 года6 Туризм.  Стандартный пакет, у меня и мужа справки с работы, оплаченное проживание, билеты через Стамбул, у меня справка о движении средств больше 2 млн за 3 месяца, у мужа накопительный счёт. На детей от меня спонсорское. На мужа учредительн

In [141]:
print(pd.Series(raw_history).to_string())

0                              виза Германии на пол года
1                         Годовые чешская и две немецкие
2                                         Испания, 1 год
3                           Чехия полгода, Германия год.
4                                            Не было виз
5      05(пт), готово было 18.05(вт))4. Германия 3 ме...
6                                Германия на месяц, год.
7      Была польская двухгодичная и много польских до...
8                                      Германия 3 месяца
9           До этого была 3х летная, немецкая, гостевая.
10                                           Не было виз
11     Виза Германия на 3 месяца в пандемию, неисполь...
12                09.214. Германия, 2-х летняя, гостевая
13                                      Польская рабочая
14        08.2021 запросили доп. документы*, оправила, 0
15                               Испанская виза на 1 год
16                                        Франция, 5 лет
17                             

In [147]:
history = []

for id, text in enumerate(raw_history):
    if (
        'нет' in text.lower()
        ):
        history.append("Нет")
        
    elif (
        'лет' in text.lower()
        ):
        history.append('Годовые и больше')
        
    elif (
        'пол' not in text.lower() and
        'год' in text.lower()
        ):
        history.append('Годовые и больше')
        
    elif (
        'пол' in text.lower()
        ):
        history.append('Полугодовые')
        
    elif (
        "мес" in text.lower()
        ):
        history.append('Меньше полугода')
        
    elif (
        'дне' in text.lower()
        ):
        history.append('Меньше полугода')
        
    elif (
        'поезд' in text.lower()
        ):
        history.append('Даты поездки')
        
    elif (
        'не' in text.lower() or
        'чист' in text.lower()
        ):
        history.append('Нет')
        
    else:
        history.append(str(id) +'_Не пон_' + text)
        
history

        



['Полугодовые',
 'Годовые и больше',
 'Годовые и больше',
 'Полугодовые',
 'Нет',
 'Меньше полугода',
 'Годовые и больше',
 'Полугодовые',
 'Меньше полугода',
 'Годовые и больше',
 'Нет',
 'Полугодовые',
 'Годовые и больше',
 'Полугодовые',
 '14_Не пон_08.2021 запросили доп. документы*, оправила, 0',
 'Годовые и больше',
 'Годовые и больше',
 'Годовые и больше',
 'Годовые и больше',
 'Годовые и больше',
 'Годовые и больше',
 'Полугодовые',
 'Полугодовые',
 'Полугодовые',
 'Годовые и больше',
 'Меньше полугода',
 'Годовые и больше',
 'Годовые и больше',
 'Годовые и больше',
 'Полугодовые',
 'Годовые и больше',
 'Годовые и больше',
 'Полугодовые',
 'Нет',
 'Годовые и больше',
 'Годовые и больше',
 'Годовые и больше',
 'Годовые и больше',
 'Полугодовые',
 'Полугодовые',
 'Годовые и больше',
 'Годовые и больше',
 'Годовые и больше',
 'Годовые и больше',
 '44_Не пон_45NaN',
 '45_Не пон_Шенген',
 'Полугодовые',
 'Полугодовые',
 'Годовые и больше',
 'Годовые и больше',
 'Годовые и больше',
 '

In [101]:
print(pd.Series(raw_limit).to_string())

0                                                  1 год
1                                                 3 года
2                                                 3 года
3                                                  1 год
4                                               3 месяца
5                                                  1 год
6                                                    Год
7                                   2 года как и просила
8                                                    Год
9                          Дали на 5 лет ( как и просил)
10                                              3 месяца
11                              1 год (просила на 1 год)
12                                                 5 лет
13                                                 1 год
14                                       На срок поездки
15                           3 месяца (просила на 1 год)
16                                                 5 лет
17                             

In [132]:
limit = []

for id, text in enumerate(raw_limit):
    if (
        'отк' in text.lower()
        ):
        limit.append("Отказ")
        
    elif (
        'лет' in text.lower()
        ):
        limit.append('Год и больше')
        
    elif (
        'пол' not in text.lower() and
        'год' in text.lower()
        ):
        limit.append('Год и больше')
        
    elif (
        'пол' in text.lower()
        ):
        limit.append('Полгода')
        
    elif (
        "мес" in text.lower()
        ):
        limit.append('Меньше полугода')
        
    elif (
        'дне' in text.lower()
        ):
        limit.append('Меньше полугода')
        
    elif (
        'поезд' in text.lower()
        ):
        limit.append('Даты поездки')
        
    else:
        limit.append(str(id) +'_Не пон_' + text)
        
limit

        


['Год и больше',
 'Год и больше',
 'Год и больше',
 'Год и больше',
 'Меньше полугода',
 'Год и больше',
 'Год и больше',
 'Год и больше',
 'Год и больше',
 'Год и больше',
 'Меньше полугода',
 'Год и больше',
 'Год и больше',
 'Год и больше',
 'Даты поездки',
 'Год и больше',
 'Год и больше',
 'Год и больше',
 'Год и больше',
 'Год и больше',
 'Год и больше',
 'Меньше полугода',
 'Отказ',
 'Год и больше',
 'Отказ',
 'Год и больше',
 'Год и больше',
 'Год и больше',
 'Год и больше',
 'Год и больше',
 'Год и больше',
 'Год и больше',
 'Год и больше',
 'Даты поездки',
 'Год и больше',
 'Год и больше',
 'Год и больше',
 'Год и больше',
 'Полгода',
 'Меньше полугода',
 'Год и больше',
 'Год и больше',
 'Полгода',
 'Год и больше',
 '44_Не пон_45NaN',
 'Меньше полугода',
 'Год и больше',
 'Год и больше',
 'Год и больше',
 'Год и больше',
 'Год и больше',
 'Отказ',
 'Год и больше',
 'Год и больше',
 'Меньше полугода',
 'Отказ',
 'Год и больше',
 'Год и больше',
 'Год и больше',
 'Год и больше

In [135]:
df = pd.DataFrame()

df['date'] = date
df['city'] = city
df['visa_center'] = visa_center
df['raw_history'] = raw_history
df['raw_limit'] = raw_limit
df

,date,city,visa_center,visa_history,raw_limit
0,28 янв 2021,Москва,ВЦ Визаметрик,виза Германии на пол года,1 год
1,09 мар 2021,Екатеринбург,ВЦ Визаметрик,Годовые чешская и две немецкие,3 года
2,26 мар 2021,Москва,ВЦ Визаметрик,"Испания, 1 год",3 года
3,19 май 2021,Москва,Визаметрик,"Чехия полгода, Германия год.",1 год
4,22 май 2021,Москва,Визаметрик,Не было виз,3 месяца
...,...,...,...,...,...
150,151NaN,Москва,ВЦ Visametric,"08 подала документы, 29.08 уведомление о готов...","1 год, запрашивала также на год"
151,152NaN,Москва,ВЦ Visametric,"Шенген (Германия) - 5 лет, 4 года (по сроку па...",5 лет (как просила)
152,153NaN,Москва,Визаметрик,"3 испанских визы 6 мес, 1 и 2 года","3 года, запрашивали 2"
153,154NaN,Калининград,ВЦ,4 летняя немецкая туристическая,5 лет


In [ ]:
df.isna().sum()